In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import json

model = SentenceTransformer("all-MiniLM-L6-v2")

faq_data = {
    "How does the return policy work?": "You can return most items within 30 days of purchase with a receipt.",
    "What are your store hours this weekend?": "We are open from 9 AM to 6 PM on both Saturday and Sunday.",
    "Can I track my order online?": "Yes, use the tracking link provided in your shipping confirmation email.",
    "How do I reset my password?": "Click 'Forgot Password?' on the login page and follow the email instructions."
}

faq_questions = list(faq_data.keys())
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)
similarity_threshold = 0.6


def handler(request):
    try:
        body = request.get_json()
        user_query = body.get("query", "")

        query_embedding = model.encode(user_query, convert_to_tensor=True)
        cosine_scores = util.cos_sim(query_embedding, faq_embeddings)

        best_match_index = torch.argmax(cosine_scores).item()
        best_score = cosine_scores[0, best_match_index].item()

        if best_score >= similarity_threshold:
            answer = faq_data[faq_questions[best_match_index]]
        else:
            answer = "Sorry, I couldn't find a relevant answer."

        return {
            "statusCode": 200,
            "headers": {"Content-Type": "application/json"},
            "body": json.dumps({"answer": answer})
        }

    except Exception as e:
        return {
            "statusCode": 500,
            "body": json.dumps({"error": str(e)})
        }


C:\Users\cdant\anaconda3\envs\jupyter_clean\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
